In [1]:
#Instalacion de dependencias

!pip install "snowflake-connector-python[pandas]"
!pip install "snowflake-connector-python[secure-local-storage,pandas]"
!pip install --upgrade snowflake-sqlalchemy

In [2]:
import warnings
warnings.filterwarnings("ignore")

from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

engine = create_engine(URL(
    account = 'pypoxiq-om82528',
    user = 'BOOTCAMPDA910',
    password = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
    database = 'ecommerce',
    schema = 'RAW',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
    #session_parameters={}
))

try:
    connection = engine.connect()
    results = connection.execute('select current_version()').fetchone()
    print('Conectado a Snowflake v{}!'.format(results[0]) )
finally:
    connection.close()
    engine.dispose()

Conectado a Snowflake v7.34.0!


In [3]:
from snowflake.sqlalchemy import URL
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
import pandas as pd
 

def fix_date_cols(df, tz='UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)
    return df

def load_csv(file, enc = 'utf-8'):
    full_url = 'ecommerce/' + file.lower()
    df = pd.read_csv(full_url, encoding=enc)
    
    if file == 'order_items.csv':
        df['shipping_limit_date']= pd.to_datetime(df['shipping_limit_date'])
        df = fix_date_cols(df)

    if file == 'order_reviews.csv':
        df['review_creation_date']= pd.to_datetime(df['review_creation_date'])
        df['review_answer_timestamp']= pd.to_datetime(df['review_answer_timestamp'])
        df = fix_date_cols(df)
        
    if file == 'orders.csv':
        df['order_purchase_timestamp']= pd.to_datetime(df['order_purchase_timestamp'])
        df['order_approved_at']= pd.to_datetime(df['order_approved_at'])
        df['order_delivered_carrier_date']= pd.to_datetime(df['order_delivered_carrier_date'])
        df['order_delivered_customer_date']= pd.to_datetime(df['order_delivered_customer_date'])
        df['order_estimated_delivery_date']= pd.to_datetime(df['order_estimated_delivery_date'])
        df = fix_date_cols(df)
    df = df.reset_index(drop=True)
    df.columns = map(lambda x: str(x).upper(), df.columns)
    table = file.lower().replace(".csv", '');
    table = table.replace(' ', '_');
    
    with engine.connect() as con:
        df.to_sql(name=table, con=con, index=False, if_exists='replace', method=pd_writer)
        print(table + ' cargada en snowflake')
        


In [4]:
load_csv('customers.csv')
load_csv('geolocation.csv')
load_csv('order_items.csv')
load_csv('order_payments.csv')
load_csv('order_reviews.csv')
load_csv('orders.csv')
load_csv('products.csv')
load_csv('sellers.csv')
load_csv('product_category_name_translation.csv')

customers cargada en snowflake
geolocation cargada en snowflake
order_items cargada en snowflake
order_payments cargada en snowflake
order_reviews cargada en snowflake
orders cargada en snowflake
products cargada en snowflake
sellers cargada en snowflake
product_category_name_translation cargada en snowflake
